In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy rustworkx scipy

# Quantum Approximate Optimization Algorithm

*Nutzungsschätzung: 22 Minutn auf am Heron r3 Prozessor (HINWEIS: Des is nur a Schätzung. Ihri Laufzeit ko variirn.)*
## Hintergrund {#Hintergrund}
Des Tutorial demonstriert d'Implementierung vom **Quantum Approximate Optimization Algorithm (QAOA)** – ana hybridn (quanten-klassischn) iterativn Methode – im Kontext vo Qiskit-Patterns. Mia wern zunächst des **Maximum-Cut** (oder **Max-Cut**) Problem für an kleinen Graphen lösen und dann lerna, wia mia des auf Utility-Skala ausführn. Olle Hardware-Ausführungen im Tutorial solln innerhalb vom Zeitlimit für den frei zugänglichn Open Plan funktionirn.

Des Max-Cut-Problem is a Optimierungsproblem, des schwer zu lösen is (genauer gsagt is es a *NP-hartes* Problem) und a Reihe verschiadener Anwendungen in Clustering, Netzwerkwissenschaft und statistischer Physik hod. Des Tutorial betrachtet an Graphen vo Knoten, de durch Kantn verbunden san, und zielt darauf ob, d'Knoten in zwoa Mengn zu partitioniern, so dass d'Anzahl da durch den Schnitt durchquerten Kantn maximiert wird.

![Illustration of a max-cut problem](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Voraussetzungen {#Voraussetzungen}

Bevor mia mit diesem Tutorial anfannga, schaugts, dass mia des Folgende installierd ham:
- Qiskit SDK v1.0 oder neuer, mit [Visualisierungs](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
- Qiskit Runtime v0.22 oder neuer (`pip install qiskit-ibm-runtime`)

Zusätzlich brauchts Zugang zu ana Instanz auf da [IBM Quantum Platform](/guides/cloud-setup). Beachts, dass des Tutorial ned im Open Plan ausgführt wern ko, weil es Workloads mit [Sessions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session) ausführt, de nur mit Premium Plan-Zugang verfügbar san.
## Einrichtung {#Einrichtung}

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Teil I. QAOA im kleinen Maßstab {#Teil-I}
Da erschte Teil vo diesem Tutorial verwendt a kloanes Max-Cut-Problem, um d'Schritte zur Lösung von am Optimierungsproblem mit am Quantencomputer z'veranschaulichn.

Um an bessern Kontext z'gebn, bevor mia des Problem auf an Quantenalgorithmus abbildn, ko mia besser verstehen, wia des Max-Cut-Problem zu am klassischn kombinatorischn Optimierungsproblem wird, wenn mia zunächst d'Minimierung von ana Funktion $f(x)$ betrachtn

$$
\min_{x\in {0, 1}^n}f(x),
$$

wobei d'Eingabe $x$ a Vektor is, des Komponentn jedem Knoten von am Graphen entsprechn. Dann wird jede vo dern Komponentn auf entweder $0$ oder $1$ beschränkt (de repräsentirn, ob sie im Schnitt enthalten san oder ned). Deser kleinskalige Beispielfall verwendt an Graphen mit $n=5$ Knotn.

Mia könntnt a Funktion von am Knotenpaar $i,j$ schreibn, de anzeigt, ob d'entsprechende Kante $(i,j)$ im Schnitt liegt. Zum Beispiel is d'Funktion $x_i + x_j - 2 x_i x_j$ genau dann 1, wenn entweder $x_i$ oder $x_j$ gleich 1 is (was bedeutet, dass d'Kante im Schnitt liegt) und sonst null. Des Problem, d'Kantn im Schnitt zu maximiern, ko formuliert wern als

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

was als Minimierung umgschrieben wern ko in da Form

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

Des Minimum vo $f(x)$ in dem Fall liegt vor, wenn d'Anzahl da durch den Schnitt durchquertn Kantn maximal is. Wia mia sehn kenna, hod des no nix mit Quantencomputing z'toa. Mia müssn des Problem in was umformuliern, was a Quantencomputer verstehen ko.
Initialisirts Euer Problem, indem mia an Graphen mit $n=5$ Knotn erstelln.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Schritt 1: Klassische Eingaben auf a Quantenproblem abbildn {#Schritt-1}

Da erschte Schritt vom Pattern bestehd darin, des klassische Problem (Graph) auf quantenmechanische **Schaltkreise** und **Operatoren** abzubildn. Dazu san drei Hauptschritte z'unternehman:

1. Verwendung von ana Reihe mathematischer Umformulierungen, um des Problem mithilfe da Notation vo Quadratic Unconstrained Binary Optimization (QUBO) Problemen darzustelln.
2. Umformulierung vom Optimierungsproblem als Hamilton-Operator, für den da Grundzustand da Lösung entspricht, de d'Kostenfunktion minimiert.
3. Erstellung von am Quantenschaltkreis, wo den Grundzustand vo dem Hamilton-Operator über an Prozess ähnlich dem Quantum Annealing vorbereitet.

**Hinweis:** In da QAOA-Methodik woin mia letztendlich an Operator (**Hamilton-Operator**) ham, wo d'**Kostenfunktion** von unserm hybridn Algorithmus darstellt, sowie an parametrisierten Schaltkreis (**Ansatz**), wo Quantenzuständ mit Kandidatenlösungen für des Problem darstellt. Mia kenna aus dern Kandidatenzuständn sampln und sie dann mit da Kostenfunktion bewertn.

#### Graph &rarr; Optimierungsproblem {#Graph-Optimierungsproblem}

Da erschte Schritt da Abbildung is a Notationsänderung. Des Folgende drückt des Problem in QUBO-Notation aus:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

wobei $Q$ a $n\times n$ Matrix reeller Zahlen is, $n$ da Anzahl da Knotn in Euerm Graphen entspricht, $x$ da obn eingeführte Vektor binärer Variablen is und $x^T$ d'Transponierte vom Vektor $x$ bezeichnet.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Optimierungsproblem &rarr; Hamilton-Operator {#Optimierungsproblem-Hamilton-Operator}

Mia kenna dann des QUBO-Problem als **Hamilton-Operator** umformuliern (do a Matrix, de d'Energie von am System darstellt):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Umformulierungsschritte vom QAOA-Problem zum Hamilton-Operator**
</summary>

Um z'demonstriern, wia des QAOA-Problem auf dera Weis umgschrieben wern ko, ersetzt mia zunächst d'binären Variablen $x_i$ durch an neien Satz vo Variablen $z_i\in{-1, 1}$ über

$$
x_i = \frac{1-z_i}{2}.
$$

Do kenna mia sehn, dass wenn $x_i$ gleich $0$ is, dann $z_i$ gleich $1$ sei muass. Wenn d'$x_i$ durch d'$z_i$ im Optimierungsproblem ($x^TQx$) ersetzt wern, ko a äquivalente Formulierung erhaitn wern.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Wenn mia etzt $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ definiern, den Vorfaktor entfernern und den konstantn Term $n^2$ weglassn, kriagn mia d'zwoa äquivalentn Formulierungen vom selbn Optimierungsproblem.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Do hängt $b$ vo $Q$ ab. Beachts, dass mia zur Erlangung vo $z^TQz + b^Tz$ den Faktor 1/4 und an konstantn Offset vo $n^2$ wegglassn ham, de bei da Optimierung koa Rolle spieln.

Um etzt a Quantenformulierung vom Problem z'kriagn, hebn mia d'Variablen $z_i$ zu ana Pauli $Z$ Matrix, wia ana $2\times 2$ Matrix da Form

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Wenn mia dene Matrizen im obign Optimierungsproblem einsetzn, kriagn mia den folgendn Hamilton-Operator

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Beachts a, dass d'$Z$ Matrizen in den Rechnenraum vom Quantencomputer einbettet san, des hoast in an Hilbert-Raum da Größe $2^n\times 2^n$. Daher solltets Terme wia $Z_iZ_j$ als des Tensorprodukt $Z_i\otimes Z_j$ verstehen, des in den $2^n\times 2^n$ Hilbert-Raum einbettet is. Zum Beispiel wird in am Problem mit fünf Entscheidungsvariablen da Term $Z_1Z_3$ als $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ verstandn, wobei $I$ d'$2\times 2$ Einheitsmatrix is.*
</details>

Deser Hamilton-Operator wird als **Kostenfunktions-Hamilton-Operator** bezeichnet. Er hod d'Eigenschaft, dass sei Grundzustand da Lösung entspricht, de d'**Kostenfunktion $f(x)$ minimiert**.
Um Euer Optimierungsproblem zu lösen, müssn mia etzt den Grundzustand vo $H_C$ (oder an Zustand mit hoher Überlappung damit) auf dem Quantencomputer präpariern. Des Sampeln aus dem Zustand wird dann mit hoher Wahrscheinlichkeit d'Lösung zu $min~f(x)$ liefern.
Schaugn mia uns etzt den Hamilton-Operator $H_C$ für des **Max-Cut** Problem an. Jedem Knotn vom Graphen wird a Qubit im Zustand $|0\rangle$ oder $|1\rangle$ zugeordnet, wobei da Wert d'Menge angibt, zu da da Knotn ghört. Des Ziel vom Problem is es, d'Anzahl da Kantn $(v_1, v_2)$ zu maximiern, für de $v_1 = |0\rangle$ und $v_2 = |1\rangle$ gilt, oder umgekehrt. Wenn mia den $Z$ Operator jedem Qubit zuordnn, wobei

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

dann ghört a Kante $(v_1, v_2)$ zum Schnitt, wenn da Eigenwert vo $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$ is; mit andern Wortn, d'mit $v_1$ und $v_2$ assoziierten Qubits san unterschiedlich. Ebenfalls ghört $(v_1, v_2)$ ned zum Schnitt, wenn da Eigenwert vo $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$ is. Beachts, dass uns da genaue Qubit-Zustand, wo jedem Knotn zugeordnet is, ned interessiert, sondern nur, ob sie über a Kante hinweg gleich san oder ned. Des Max-Cut-Problem verlangt vo uns, a Zuordnung da Qubits auf d'Knotn z'findn, so dass da Eigenwert vom folgendn Hamilton-Operator minimiert wird
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

Mit andern Wortn, $b_i = 0$ für olle $i$ im Max-Cut-Problem. Da Wert vo $Q_{ij}$ bezeichnet des Gewicht da Kante. In dem Tutorial betrachtn mia an ungewichteten Graphen, des hoast $Q_{ij} = 1.0$ für olle $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Schritt 2: Problem für Quanten-Hardware-Ausführung optimiern {#Schritt-2}
Da obige Schaltkreis enthält a Reihe vo Abstraktionen, de nützlich san, um über Quantenalgorithmen nachzudenken, aber auf da Hardware ned ausführbar san. Um auf ana QPU ausgführt wern z'kenna, muass da Schaltkreis a Reihe vo Operationen durchlaufn, de den **Transpilations**- oder **Schaltkreis-Optimierungs**-Schritt vom Pattern ausmachn.

D'Qiskit-Bibliothek bietet a Reihe vo **Transpilations-Pässen**, de a breite Palette vo Schaltkreistransformationen abdeckn. Mia müssn sicherstellen, dass Euer Schaltkreis für Euerern Zweck **optimiert** is.

D'Transpilation ko mehrere Schritte umfassn, wia zum Beispiel:

* **Initiales Mapping** da Qubits im Schaltkreis (wia Entscheidungsvariablen) auf physische Qubits auf dem Gerät.
* **Unrolling** da Anweisungen im Quantenschaltkreis zu den hardware-nativen Anweisungen, de des Backend verstehd.
* **Routing** beliebiger Qubits im Schaltkreis, wo interagiern, zu physischen Qubits, de benachbart zueinanda san.
* **Fehlerunterdrückung** durch Hinzufügn vo Einzelqubit-Gates zur Rauschunterdrückung mit dynamischer Entkopplung.

Weitere Informationen zur Transpilation findets in unserer [Dokumentation](/guides/transpile).

Da folgende Code transformiert und optimiert den abstraktn Schaltkreis in a Format, des zur Ausführung auf am da über d'Cloud zugänglichn Geräte mit dem **Qiskit IBM Runtime Service** bereit is.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Schritt 3: Ausführung mit Qiskit Primitives {#Schritt-3}
Im QAOA-Workflow wern d'optimalen QAOA-Parameter in ana iterativen Optimierungsschleife gfundn, de a Reihe vo Schaltkreisbewertungen ausführt und an klassischn Optimierer verwendt, um d'optimalen $\beta_k$ und $\gamma_k$ Parameter z'findn. Dese Ausführungsschleife wird über d'folgendn Schritte ausgführt:

1. Definierung da initialen Parameter
2. Instanziierung von ana neien `Session`, de d'Optimierungsschleife und des Primitive enthält, wo zum Sampln vom Schaltkreis verwendt wird
3. Sobald a optimaler Parametersatz gfundn is, führts den Schaltkreis a letztns Mal aus, um a finale Verteilung z'kriagn, de im Post-Processing-Schritt verwendt wird.
#### Schaltkreis mit initialen Parametern definiern {#Schaltkreis-initialen-Parameter}
Mia fanng mit willkürlich gwahltn Parametern an.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Backend und Ausführungs-Primitive definiern {#Backend-Primitive}
Verwendts d'**Qiskit Runtime Primitives**, um mit IBM&reg; Backends z'interagiern. D'zwoa Primitives san Sampler und Estimator, und d'Wahl vom Primitive hängt davon ob, welche Art vo Messung mia auf dem Quantencomputer ausführn woin. Für d'Minimierung vo $H_C$ verwendts den Estimator, weil d'Messung da Kostenfunktion einfach da Erwartungswert vo $\langle H_C \rangle$ is.
#### Ausführn {#Ausfuehrn}

D'Primitives bietn a Vielzahl vo [Ausführungsmodi](/guides/execution-modes) zur Planung vo Workloads auf Quantengeräten, und a QAOA-Workflow läuft iterativ in ana Session.

![Illustration showing the behavior of Single job, Batch, and Session runtime modes.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

Mia kenna d'sampler-basierte Kostenfunktion in d'SciPy-Minimierungsroutine eistecka, um d'optimalen Parameter z'findn.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Sobald mia d'optimalen Parameter für den Schaltkreis gfundn ham, kenna mia dene Parameter zuweisn und d'mit den optimierten Parametern erhaltene finale Verteilung sampln. Do sollt des *Sampler* Primitive verwendt wern, weil es d'Wahrscheinlichkeitsverteilung vo Bitstring-Messungen is, de dem optimalen Schnitt vom Graphen entsprechn.

**Hinweis:** Des bedeutet, an Quantenzustand $\psi$ im Computer z'präpariern und ihn dann z'messn. A Messung wird den Zustand in an einzelnen Berechnungsbasiszustand kollabiern - zum Beispiel `010101110000...` - wo ana Kandidatenlösung $x$ für unser ursprünglichs Optimierungsproblem ($\max f(x)$ oder $\min f(x)$ je nach Aufgabe) entspricht.

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Schritt 4: Nachbearbeitung und Rückgabe vom Ergebnis im gwünschten klassischn Format {#Schritt-4}

Da Nachbearbeitungsschritt interpretierd d'Sampling-Ausgabe, um a Lösung für Euer ursprünglichs Problem zurückzugeben. In dem Fall san mia an dem Bitstring mit da höchstn Wahrscheinlichkeit interessiert, weil deser den optimalen Schnitt bestimmt. D'Symmetrien im Problem erlaubn vier mögliche Lösungen, und da Sampling-Prozess wird a davon mit ana etwas höhern Wahrscheinlichkeit zurückgebn, aber mia kenna in da untn dargestelltn Verteilung sehn, dass vier da Bitstrings deutlich wahrscheinlicher san als da Rest.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Besten Schnitt visualisiern {#Besten-Schnitt}

Aus dem optimalen Bitstring kenna mia dann den Schnitt auf dem ursprünglichn Graphen visualisiern.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

Und berechnt den Wert vom Schnitt:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Teil II. Skalierng auffe! {#Teil-II}

Mia ham Zugang zu vüln Gerätn mit über 100 Qubits auf da IBM Quantum&reg; Platform. Wählts oas aus, auf dem Max-Cut auf am 100-Knoten-gewichteten Graphen glöst wern soll. Des is a Problem im "Utility-Maßstab". D'Schritte zum Aufbau vom Workflow san d'gleichen wia obn, jedoch mit am vül größern Graphen.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Schritt 1: Klassische Eingaben auf a Quantenproblem abbildn {#Schritt-1-II}
#### Graph &rarr; Hamilton-Operator {#Graph-Hamilton-Operator-II}
Konvertierst zunächst den Graphen, den mia lösen woin, direkt in an Hamilton-Operator, wo für QAOA geignet is.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamilton-Operator &rarr; Quantenschaltkreis {#Hamilton-Operator-Quantenschaltkreis-II}

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Schritt 2: Problem für Quantenausführung optimiern {#Schritt-2-II}
Um den Schaltkreis-Optimierungsschritt auf Utility-Scale-Probleme zu skailiern, kenna mia d'in Qiskit SDK v1.0 eingeführten Hochleistungs-Transpilationsstrategien nutzn. Weitere Werkzeuge umfassn den neien Transpiler-Service mit [KI-erweiterten Transpiler-Pässen](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Sobald d'optimalen Parameter aus da Ausführung vo QAOA auf dem Gerät gfundn wurn, weists d'Parameter dem Schaltkreis zu.